# Direct Radio Poll Test

In [ ]:
%autosave 0

In [ ]:
from __future__ import print_function
from builtins import *                  # python3 types
from time import sleep
import struct as pystruct
from binascii import hexlify
import sys
import os.path
import datetime

In [ ]:
sys.path.insert(0,"../tagnet")
from tagnet.tagmessages import TagMessage, TagPoll, TagGet, TagPut, TagDelete, TagHead
from tagnet.tagnames import TagName
from tagnet.tagtlv import TagTlv, TagTlvList, tlv_types

In [ ]:
sys.path.insert(0,"../tagfuse")
sys.path.insert(0,"../si446x/")
from tagfuse.radioutils import payload2values, radio_receive_msg, radio_send_msg
from tagfuse.radioutils import radio_start, radio_config, radio_show_config

In [ ]:
sys.path

In [ ]:
from tagfuse.tagfuseargs import __version__
'tagfuse', __version__

In [ ]:
print('Test Start Time: {}'.format(datetime.datetime.now()))
#print('Si446x Radio Device Driver Version: {}'.format(si446x_device_version()))

##  Start up Radio

In [ ]:
radio = radio_start()

In [ ]:
radio_show_config(radio.dump_radio())

### Set Defaults

In [ ]:
# default paramters
MAX_WAIT            = 1
MAX_RECV            = 255
MAX_PAYLOAD         = 254
MAX_RETRIES         = 10
RADIO_POWER         = 10
SHORT_DELAY         = 1

### Poll for Tags

In [ ]:
STOP


In [ ]:
def send_poll():
    req_obj = TagPoll() # sends time, slot_time, slot_count, node_id, node_name
    req_msg = req_obj.build()
    return radio_send_msg(radio, req_msg, RADIO_POWER)

In [ ]:
send_poll()

In [ ]:
from datetime import datetime
from uuid import getnode as get_mac
import platform
slot_time=100
slot_count=10
pl = TagTlvList([
    #[(tlv_types.TIME,datetime.now()),])
                         (tlv_types.INTEGER,slot_time),
                         (tlv_types.INTEGER,slot_count),
                         (tlv_types.NODE_ID, get_mac()),
                         (tlv_types.NODE_NAME, platform.node())])
type(pl), pl

In [ ]:
TagTlv(tlv_types.TIME,0)

In [ ]:
def poll_for_tags():
    req_obj = TagPoll() # sends time, slot_time, slot_count, node_id, node_name
    req_msg = req_obj.build()
    sstatus = radio_send_msg(radio, req_msg, RADIO_POWER)
    rsp_msg, rssi, rstatus = radio_receive_msg(radio, MAX_RECV, 3)
    if rsp_msg:
        #        print(hexlify(rsp_msg))
        rsp_obj = TagMessage(rsp_msg)
        #        print(rsp_obj.header.options.param.error_code)
        #        print(rsp_obj.payload)
        # print(rsp_obj.header, rsp_obj.name)
        if rsp_obj.payload:
            return rsp_obj.payload, rssi, sstatus, rstatus
        return none, rssi, sstatus, rstatus
    return None, None, sstatus, rstatus

In [ ]:
payload, rssi, sstatus, rstatus = poll_for_tags()
payload, rssi

In [ ]:
for i in range(10000):
    payload, rssi, sstatus, rstatus = poll_for_tags()
    print(payload, rssi)

In [ ]:
import platform
platform.node()

In [ ]:
TagTlv(tlv_types)

In [ ]:
rssi, payload

In [ ]:
'send', sstatus, 'receive', rstatus

In [ ]:
sstatus = radio_send_msg(radio, req_msg, RADIO_POWER)

In [ ]:
rsp_msg, rssi, rstatus = radio_receive_msg(radio, MAX_RECV, MAX_WAIT)
rsp_msg, rssi

In [ ]:
radio.trace.display(radio.trace.filter(count=-10))

## Get Tag's RSSI value

In [ ]:
STOP

In [ ]:
def tag_get_rssi():
    name = TagName([TagTlv(tlv_types.NODE_ID, -1),
                            TagTlv('tag'),
                            TagTlv('.test'),
                            TagTlv('rssi'),
                             ])
    req_obj = TagGet(name)
    req_msg = req_obj.build()
    sstatus = radio_send_msg(radio, req_msg, RADIO_POWER)
    rsp_msg, rssi, rstatus = radio_receive_msg(radio, MAX_RECV, 3)
    if rsp_msg:
        #        print(hexlify(rsp_msg))
        rsp_obj = TagMessage(rsp_msg)
        #        print(rsp_obj.header.options.param.error_code)
        #        print(rsp_obj.payload)
        # print(rsp_obj.header, rsp_obj.name)
        if rsp_obj.payload:
            return rsp_obj.payload[0].value(), rssi, sstatus, rstatus
        return none, rssi, sstatus, rstatus
    return None, None, sstatus, rstatus

In [ ]:
tag_get_rssi()

In [ ]:
RADIO_POWER=2
for z in range(100):
    for RADIO_POWER in range(1,122,10):
        for i in range(2):
            remote_rssi, rssi, _, _ = tag_get_rssi()
            if rssi == -1:
                break
            print('{:>4}.{:<4} power: {:3},   rssi l/r:  {:4>}/{:<4}'.format(z,i, RADIO_POWER, rssi, remote_rssi))

## Get Tag's Transmit Power value

In [ ]:
STOP

In [ ]:
def tag_get_power():
    name = TagName([TagTlv(tlv_types.NODE_ID, -1),
                            TagTlv('tag'),
                            TagTlv('.test'),
                            TagTlv('tx_pwr'),
                             ])
    req_obj = TagGet(name)
    req_msg = req_obj.build()
    sstatus = radio_send_msg(radio, req_msg, RADIO_POWER)
    rsp_msg, rssi, rstatus = radio_receive_msg(radio, MAX_RECV, 3)
    if rsp_msg:
        #        print(hexlify(rsp_msg))
        rsp_obj = TagMessage(rsp_msg)
        #        print(rsp_obj.header.options.param.error_code)
        #        print(rsp_obj.payload)
        print(rsp_obj.header, rsp_obj.name)
        if rsp_obj.payload:
            print(rsp_obj.payload)
        return rsp_obj.payload[0].value(), rssi, sstatus, rstatus
    return None, None, sstatus, rstatus

In [ ]:
tag_get_power()

In [ ]:
radio.spi.trace.display(radio.spi.trace.filter(count=-100))

## Set Tag's Transmit Power value

In [ ]:
STOP

In [ ]:
def tag_set_power(power):
    name = TagName([TagTlv(tlv_types.NODE_ID, -1),
                            TagTlv('tag'),
                            TagTlv('.test'),
                            TagTlv('tx_pwr'),
                             ])
    req_obj = TagPut(name, pl=TagTlvList([TagTlv(tlv_types.INTEGER, power)]))
    req_msg = req_obj.build()
    sstatus = radio_send_msg(radio, req_msg, RADIO_POWER)
    rsp_msg, rssi, rstatus = radio_receive_msg(radio, MAX_RECV, 3)
    if rsp_msg:
        #        print(hexlify(rsp_msg))
        rsp_obj = TagMessage(rsp_msg)
        #        print(rsp_obj.header.options.param.error_code)
        #        print(rsp_obj.payload)
        #print(rsp_obj.header, rsp_obj.name)
        #if rsp_obj.payload:
        #    print(rsp_obj.payload)
        return rsp_obj.payload, rssi, sstatus, rstatus
    return None, None, sstatus, rstatus

In [ ]:
for z in range(10000):
    for TAG_RADIO_POWER in range(1,128,10):
        for i in range(6):
            _, rssi, _, _ = tag_set_power(TAG_RADIO_POWER)
            tag_rssi, rssi, _, _ = tag_get_rssi()
            if rssi == -1:
                break
            print('{:>4}.{:<4} power: {:3},   rssi l/r:  {:4>}/{:<4}'.format(z,i, TAG_RADIO_POWER, rssi, tag_rssi))

In [ ]:
TAG_RADIO_POWER=255

In [ ]:
tag_set_power(TAG_RADIO_POWER)

In [ ]:
radio.spi.trace.display(radio.spi.trace.filter(count=-100))

### Write Zeros

In [ ]:
STOP

In [ ]:
#<node_id>   "tag"  "test"   "zero"   "byte"
def read_test_bytes(testfile, pos, num):
    zero_byte_name = TagName([TagTlv(tlv_types.NODE_ID, -1),
                            TagTlv('tag'),
                            TagTlv('.test'),
                            TagTlv(testfile),
                            TagTlv('byte'),
                            TagTlv(tlv_types.OFFSET, pos),
                            TagTlv(tlv_types.SIZE, num),
                             ])
    get_zero_byte = TagGet(zero_byte_name)
#    print(get_gps_xyz.name)
    req_msg = get_zero_byte.build()
    radio_send_msg(radio, req_msg, RADIO_POWER);
    rsp_msg, rssi, status = radio_receive_msg(radio, MAX_RECV, MAX_WAIT)
    if rsp_msg:
#        print(hexlify(rsp_msg))
        rsp_obj = TagMessage(rsp_msg)
#        print(rsp_obj.header.options.param.error_code)
#        print(rsp_obj.payload)
        if rsp_obj.payload:
            error, offset, amt, block = payload2values(rsp_obj.payload,
                                  [tlv_types.ERROR,
                                   tlv_types.OFFSET,
                                   tlv_types.SIZE,
                                   tlv_types.BLOCK,
                                  ])
            seta = set(block)
            # print(seta)
            if (len(seta) > 1):
                print('check error', seta, amt, hexlify(block))
                amt = 0
            return error, offset, amt, block
        else:
            print("{}".format(rsp_obj.header.options.param.error_code))
#    else:
#        print('TIMEOUT')
    return None

In [ ]:
start = datetime.datetime.now()
for x in range(1,1000):
    for y in range(1,10):
        sleep(SHORT_DELAY)
        return_values = read_test_bytes('zero', 1, 1)
        if return_values:
            error, offset, amt, block = return_values
            # print('error: {}'.format(error))
            if error and error is not tlv_errors.SUCCESS:
                print("\r{} {}".format(datetime.now() - start, error), end="")
                continue
            print('got bytes, len: {}, count: {}, offset: {}'.format(len(block),
                                                                    amt,
                                                                    offset))
        else:
            print('timeout')
            
print("\ntime:{}".format(datetime.now() - start))

## Listen for packets

In [ ]:
for i in range(200000):
    rsp_msg, rssi, status = radio_receive_msg(radio, MAX_RECV, 3000)
    if rsp_msg:
        print(datetime.datetime.now(), rssi, len(rsp_msg), hexlify(rsp_msg))
        #rsp_obj = TagMessage(rsp_msg)
        #        print(rsp_obj.header.options.param.error_code)
        #        print(rsp_obj.payload)
        #if rsp_obj.payload:
        #    print(rsp_obj.header, rsp_obj.name, rsp_obj.payload)

## Get Chip Status

In [ ]:
STOP

In [ ]:
print(radio.get_chip_status())

## Interactive Group Properties

In [ ]:
STOP

In [ ]:
interact(si446x_device_group_fetch_and_decode, group=radio_config_group_ids.encoding)

## Interactive  Command Status Responses

In [ ]:
interact(si446x_device_command_fetch_and_decode, cmd=radio_status_cmd_ids.encoding)

In [ ]:
from datetime import datetime
datetime.now()